In [1]:
import os
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from scipy.stats import entropy, kurtosis
import xgboost as xgb
import seaborn as sns
from xgboost import plot_importance
import warnings
import matplotlib.pyplot as plt
import pandas as pd
from math import *
import numpy as np

pd.set_option('display.max_columns', None)
from IPython.display import display
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
pathf = os.path.join("..", "data", "particles")

trainpd = pd.read_csv(os.path.join(pathf, "train.csv"))
print(trainpd.head(1))
trainshape = trainpd.shape
print(trainshape)
eventpd = pd.read_csv(os.path.join(pathf, "event.csv"))
print(eventpd.head(1))
print(eventpd.shape)
testpd = pd.read_csv(os.path.join(pathf, "test.csv"))
testshape = testpd.shape
print(testpd.head(1))
print(testpd.shape)

data = pd.concat([trainpd, testpd], ignore_index=True)
data = pd.merge(data, eventpd, on='event_id', how='left')


       x      y  z        t   terror        q  flag  event_id  hit_id
0 -142.5 -147.5  0  767.879  2.02966  1.05052     0         7       1
(9473201, 9)
   event_id  nhit  nhitreal  energymc  thetamc    phimc   xcmc    ycmc
0         7   426        70   48348.9  63.1686  11.0982 -40.83  114.03
(13315, 8)
       x      y  z        t  terror        q  event_id  hit_id
0 -142.5 -127.5  0  848.061  1.9984  1.15067         9       1
(4086511, 8)


In [3]:
# (k(q,mc)*(t0+l))^2 + dis^2 -dis*cos(phi)*sin(thmc)*(t0+l) = (t+l)^2
data['fx'] = data['x'] - data['xcmc']
data['fy'] = data['y'] - data['ycmc']
data['phimc'] = data['phimc'] * np.pi / 180.
data['fphi'] = np.arctan2(data['fy'], data['fx']) - data['phimc']
data['fdis'] = np.sqrt(data['fx'] ** 2 + data['fy'] ** 2)
data['thetamc'] = data['thetamc'] * np.pi / 180.
data['fsinthmc'] = np.sin(data['thetamc'])
data['fcosphi'] = np.cos(data['fphi'])

data['ft2'] = data['t'] ** 2
data['fdis2'] = data['fdis'] ** 2
data['fsencond'] = data['fdis'] * data['fcosphi'] * data['fsinthmc']

data['fttrue'] = data['t'] / data['terror']
data['nhitratio'] = data['nhit'] / data['nhitreal']

data['fenergymc2'] = data['energymc'] ** 2

del data['fx']
del data['fy']
del data['x']
del data['y']
del data['z']


In [4]:
info_new = pd.DataFrame()
info_new["event_id"] = data.groupby(["event_id"])["event_id"].mean()
info_new["fdis_mean"] = data.groupby(["event_id"])["fdis"].mean()
info_new["fdis_std"] = data.groupby(["event_id"])["fdis"].std()
info_new["fdis_stdmean"] = info_new["fdis_std"] / info_new["fdis_mean"]
info_new["ft_mean"] = data.groupby(["event_id"])["t"].mean()
info_new["ft_std"] = data.groupby(["event_id"])["t"].std()
info_new["ft_stdmean"] = info_new["ft_std"] / info_new["ft_mean"]
info_new["ft_mean2"] = info_new["ft_mean"] ** 2
info_new.reset_index(drop=True, inplace=True)
data = pd.merge(data, info_new, on='event_id', how='left')

data['fsencond2'] = data['fsencond'] * data['ft_mean']


In [5]:
print(trainshape[0])
print(data.shape)

trainpd = data[:trainshape[0]].reset_index()
testpd = data[trainshape[0]:].reset_index()
del data

9473201
(13559712, 31)


In [6]:
print(trainpd.columns)
feature = [x for x in trainpd.columns if x not in ['flag', 'index', 'hit_id', 'event_id']]
labels = trainpd['flag']
del trainpd['flag']
del testpd['flag']


Index(['index', 't', 'terror', 'q', 'flag', 'event_id', 'hit_id', 'nhit',
       'nhitreal', 'energymc', 'thetamc', 'phimc', 'xcmc', 'ycmc', 'fphi',
       'fdis', 'fsinthmc', 'fcosphi', 'ft2', 'fdis2', 'fsencond', 'fttrue',
       'nhitratio', 'fenergymc2', 'fdis_mean', 'fdis_std', 'fdis_stdmean',
       'ft_mean', 'ft_std', 'ft_stdmean', 'ft_mean2', 'fsencond2'],
      dtype='object')


In [7]:
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=4399)
y_pp_xgb = np.zeros(testshape[0])
y_pp_xgb_stacking = np.zeros(len(labels))
for train_index, test_index in kf.split(trainpd):
    print(">>>", train_index)
    clf = xgb.XGBClassifier(tree_method='gpu_hist', max_depth=8, learning_rate=0.1, verbosity=3,
                            eval_metric='auc', n_estimators=2000)
    clf.fit(
        trainpd[feature].iloc[train_index], labels[train_index],
        eval_set=[(trainpd[feature].iloc[train_index], labels[train_index]),
                  (trainpd[feature].iloc[test_index], labels[test_index])],
        early_stopping_rounds=50,
        verbose=True,
    )

    y_pred = clf.predict(trainpd[feature].iloc[test_index])
    y_predprob = clf.predict_proba(trainpd[feature].iloc[test_index])[:, 1]

    y_pp_xgb_stacking[test_index] = y_predprob

    auc = metrics.roc_auc_score(labels[test_index], y_predprob)
    print("AUC Score (Train): %f" % auc)

    y_pp_xgb += clf.predict_proba(testpd[feature])[:, 1] / n_splits


>>> [      0       2       3 ... 9473198 9473199 9473200]
[10:15:28] DEBUG: /workspace/src/tree/updater_gpu_hist.cu:1167: [GPU Hist]: Configure
[10:15:29] DEBUG: /workspace/src/common/device_helpers.cu:38: Running nccl init on: 9.0
[10:15:29] ======== Monitor: DenseCuts ========
[10:15:29] Init: 0.004996s, 1 calls @ 4996us

[10:15:30] ======== Monitor: HistogramCuts ========
[0]	validation_0-auc:0.96971	validation_1-auc:0.96942
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 50 rounds.
[1]	validation_0-auc:0.97234	validation_1-auc:0.97216
[2]	validation_0-auc:0.97494	validation_1-auc:0.97481
[3]	validation_0-auc:0.97650	validation_1-auc:0.97645
[4]	validation_0-auc:0.97696	validation_1-auc:0.97692
[5]	validation_0-auc:0.97872	validation_1-auc:0.97866
[6]	validation_0-auc:0.97966	validation_1-auc:0.97959
[7]	validation_0-auc:0.98013	validation_1-auc:0.98004
[8]	validation_0-auc:0.98067	vali

[139]	validation_0-auc:0.99567	validation_1-auc:0.99552
[140]	validation_0-auc:0.99569	validation_1-auc:0.99555
[141]	validation_0-auc:0.99570	validation_1-auc:0.99556
[142]	validation_0-auc:0.99570	validation_1-auc:0.99556
[143]	validation_0-auc:0.99571	validation_1-auc:0.99557
[144]	validation_0-auc:0.99575	validation_1-auc:0.99560
[145]	validation_0-auc:0.99580	validation_1-auc:0.99565
[146]	validation_0-auc:0.99582	validation_1-auc:0.99567
[147]	validation_0-auc:0.99584	validation_1-auc:0.99569
[148]	validation_0-auc:0.99586	validation_1-auc:0.99570
[149]	validation_0-auc:0.99587	validation_1-auc:0.99572
[150]	validation_0-auc:0.99588	validation_1-auc:0.99572
[151]	validation_0-auc:0.99589	validation_1-auc:0.99574
[152]	validation_0-auc:0.99591	validation_1-auc:0.99576
[153]	validation_0-auc:0.99595	validation_1-auc:0.99579
[154]	validation_0-auc:0.99597	validation_1-auc:0.99581
[155]	validation_0-auc:0.99598	validation_1-auc:0.99582
[156]	validation_0-auc:0.99598	validation_1-auc:

[286]	validation_0-auc:0.99712	validation_1-auc:0.99682
[287]	validation_0-auc:0.99712	validation_1-auc:0.99682
[288]	validation_0-auc:0.99712	validation_1-auc:0.99682
[289]	validation_0-auc:0.99713	validation_1-auc:0.99683
[290]	validation_0-auc:0.99713	validation_1-auc:0.99683
[291]	validation_0-auc:0.99713	validation_1-auc:0.99683
[292]	validation_0-auc:0.99713	validation_1-auc:0.99683
[293]	validation_0-auc:0.99713	validation_1-auc:0.99683
[294]	validation_0-auc:0.99714	validation_1-auc:0.99684
[295]	validation_0-auc:0.99715	validation_1-auc:0.99684
[296]	validation_0-auc:0.99715	validation_1-auc:0.99684
[297]	validation_0-auc:0.99715	validation_1-auc:0.99685
[298]	validation_0-auc:0.99715	validation_1-auc:0.99685
[299]	validation_0-auc:0.99715	validation_1-auc:0.99685
[300]	validation_0-auc:0.99716	validation_1-auc:0.99685
[301]	validation_0-auc:0.99717	validation_1-auc:0.99686
[302]	validation_0-auc:0.99718	validation_1-auc:0.99687
[303]	validation_0-auc:0.99718	validation_1-auc:

[433]	validation_0-auc:0.99760	validation_1-auc:0.99718
[434]	validation_0-auc:0.99760	validation_1-auc:0.99718
[435]	validation_0-auc:0.99761	validation_1-auc:0.99718
[436]	validation_0-auc:0.99761	validation_1-auc:0.99718
[437]	validation_0-auc:0.99761	validation_1-auc:0.99718
[438]	validation_0-auc:0.99761	validation_1-auc:0.99718
[439]	validation_0-auc:0.99761	validation_1-auc:0.99718
[440]	validation_0-auc:0.99762	validation_1-auc:0.99718
[441]	validation_0-auc:0.99762	validation_1-auc:0.99719
[442]	validation_0-auc:0.99762	validation_1-auc:0.99719
[443]	validation_0-auc:0.99762	validation_1-auc:0.99719
[444]	validation_0-auc:0.99763	validation_1-auc:0.99719
[445]	validation_0-auc:0.99764	validation_1-auc:0.99720
[446]	validation_0-auc:0.99764	validation_1-auc:0.99720
[447]	validation_0-auc:0.99764	validation_1-auc:0.99720
[448]	validation_0-auc:0.99765	validation_1-auc:0.99721
[449]	validation_0-auc:0.99765	validation_1-auc:0.99721
[450]	validation_0-auc:0.99765	validation_1-auc:

[580]	validation_0-auc:0.99792	validation_1-auc:0.99736
[581]	validation_0-auc:0.99792	validation_1-auc:0.99736
[582]	validation_0-auc:0.99792	validation_1-auc:0.99736
[583]	validation_0-auc:0.99793	validation_1-auc:0.99736
[584]	validation_0-auc:0.99793	validation_1-auc:0.99736
[585]	validation_0-auc:0.99793	validation_1-auc:0.99736
[586]	validation_0-auc:0.99793	validation_1-auc:0.99736
[587]	validation_0-auc:0.99793	validation_1-auc:0.99736
[588]	validation_0-auc:0.99794	validation_1-auc:0.99736
[589]	validation_0-auc:0.99794	validation_1-auc:0.99737
[590]	validation_0-auc:0.99794	validation_1-auc:0.99737
[591]	validation_0-auc:0.99794	validation_1-auc:0.99737
[592]	validation_0-auc:0.99795	validation_1-auc:0.99737
[593]	validation_0-auc:0.99795	validation_1-auc:0.99737
[594]	validation_0-auc:0.99795	validation_1-auc:0.99737
[595]	validation_0-auc:0.99795	validation_1-auc:0.99737
[596]	validation_0-auc:0.99795	validation_1-auc:0.99737
[597]	validation_0-auc:0.99795	validation_1-auc:

...

[1151]	validation_0-auc:0.99862	validation_1-auc:0.99755
[1152]	validation_0-auc:0.99862	validation_1-auc:0.99755
[1153]	validation_0-auc:0.99862	validation_1-auc:0.99755
[1154]	validation_0-auc:0.99862	validation_1-auc:0.99755
[1155]	validation_0-auc:0.99862	validation_1-auc:0.99755
[1156]	validation_0-auc:0.99863	validation_1-auc:0.99755
[1157]	validation_0-auc:0.99863	validation_1-auc:0.99755
[1158]	validation_0-auc:0.99863	validation_1-auc:0.99755
[1159]	validation_0-auc:0.99863	validation_1-auc:0.99755
[1160]	validation_0-auc:0.99863	validation_1-auc:0.99755
[1161]	validation_0-auc:0.99863	validation_1-auc:0.99755
[1162]	validation_0-auc:0.99863	validation_1-auc:0.99755
[1163]	validation_0-auc:0.99863	validation_1-auc:0.99755
[1164]	validation_0-auc:0.99863	validation_1-auc:0.99755
[1165]	validation_0-auc:0.99864	validation_1-auc:0.99755
[1166]	validation_0-auc:0.99864	validation_1-auc:0.99755
[1167]	validation_0-auc:0.99864	validation_1-auc:0.99755
[1168]	validation_0-auc:0.99864

[1295]	validation_0-auc:0.99874	validation_1-auc:0.99759
[1296]	validation_0-auc:0.99874	validation_1-auc:0.99759
[1297]	validation_0-auc:0.99875	validation_1-auc:0.99759
[1298]	validation_0-auc:0.99875	validation_1-auc:0.99759
[1299]	validation_0-auc:0.99875	validation_1-auc:0.99759
[1300]	validation_0-auc:0.99875	validation_1-auc:0.99759
[1301]	validation_0-auc:0.99875	validation_1-auc:0.99759
[1302]	validation_0-auc:0.99875	validation_1-auc:0.99759
[1303]	validation_0-auc:0.99875	validation_1-auc:0.99759
[1304]	validation_0-auc:0.99875	validation_1-auc:0.99759
[1305]	validation_0-auc:0.99875	validation_1-auc:0.99759
[1306]	validation_0-auc:0.99875	validation_1-auc:0.99759
[1307]	validation_0-auc:0.99875	validation_1-auc:0.99759
[1308]	validation_0-auc:0.99875	validation_1-auc:0.99759
[1309]	validation_0-auc:0.99875	validation_1-auc:0.99759
[1310]	validation_0-auc:0.99875	validation_1-auc:0.99759
[1311]	validation_0-auc:0.99876	validation_1-auc:0.99759
[1312]	validation_0-auc:0.99876

[1439]	validation_0-auc:0.99884	validation_1-auc:0.99760
[1440]	validation_0-auc:0.99884	validation_1-auc:0.99760
[1441]	validation_0-auc:0.99884	validation_1-auc:0.99760
[1442]	validation_0-auc:0.99884	validation_1-auc:0.99760
[1443]	validation_0-auc:0.99885	validation_1-auc:0.99760
[1444]	validation_0-auc:0.99885	validation_1-auc:0.99760
[1445]	validation_0-auc:0.99885	validation_1-auc:0.99760
[1446]	validation_0-auc:0.99885	validation_1-auc:0.99760
[1447]	validation_0-auc:0.99885	validation_1-auc:0.99760
[1448]	validation_0-auc:0.99885	validation_1-auc:0.99760
[1449]	validation_0-auc:0.99885	validation_1-auc:0.99760
[1450]	validation_0-auc:0.99885	validation_1-auc:0.99760
[1451]	validation_0-auc:0.99885	validation_1-auc:0.99760
[1452]	validation_0-auc:0.99885	validation_1-auc:0.99760
[1453]	validation_0-auc:0.99885	validation_1-auc:0.99760
[1454]	validation_0-auc:0.99885	validation_1-auc:0.99760
[1455]	validation_0-auc:0.99885	validation_1-auc:0.99760
[1456]	validation_0-auc:0.99885

[1583]	validation_0-auc:0.99893	validation_1-auc:0.99762
[1584]	validation_0-auc:0.99893	validation_1-auc:0.99762
[1585]	validation_0-auc:0.99893	validation_1-auc:0.99762
[1586]	validation_0-auc:0.99893	validation_1-auc:0.99762
[1587]	validation_0-auc:0.99894	validation_1-auc:0.99762
[1588]	validation_0-auc:0.99894	validation_1-auc:0.99762
[1589]	validation_0-auc:0.99894	validation_1-auc:0.99762
[1590]	validation_0-auc:0.99894	validation_1-auc:0.99762
[1591]	validation_0-auc:0.99894	validation_1-auc:0.99762
[1592]	validation_0-auc:0.99894	validation_1-auc:0.99762
[1593]	validation_0-auc:0.99894	validation_1-auc:0.99762
[1594]	validation_0-auc:0.99894	validation_1-auc:0.99762
[1595]	validation_0-auc:0.99894	validation_1-auc:0.99762
[1596]	validation_0-auc:0.99894	validation_1-auc:0.99762
[1597]	validation_0-auc:0.99894	validation_1-auc:0.99762
[1598]	validation_0-auc:0.99894	validation_1-auc:0.99762
[1599]	validation_0-auc:0.99894	validation_1-auc:0.99762
[1600]	validation_0-auc:0.99894

[1727]	validation_0-auc:0.99901	validation_1-auc:0.99762
[1728]	validation_0-auc:0.99901	validation_1-auc:0.99762
[1729]	validation_0-auc:0.99901	validation_1-auc:0.99762
[1730]	validation_0-auc:0.99901	validation_1-auc:0.99762
[1731]	validation_0-auc:0.99901	validation_1-auc:0.99762
[1732]	validation_0-auc:0.99901	validation_1-auc:0.99762
[1733]	validation_0-auc:0.99901	validation_1-auc:0.99762
[1734]	validation_0-auc:0.99901	validation_1-auc:0.99762
[1735]	validation_0-auc:0.99901	validation_1-auc:0.99762
[1736]	validation_0-auc:0.99901	validation_1-auc:0.99762
[1737]	validation_0-auc:0.99901	validation_1-auc:0.99762
[1738]	validation_0-auc:0.99902	validation_1-auc:0.99762
[1739]	validation_0-auc:0.99902	validation_1-auc:0.99762
[1740]	validation_0-auc:0.99902	validation_1-auc:0.99762
[1741]	validation_0-auc:0.99902	validation_1-auc:0.99762
[1742]	validation_0-auc:0.99902	validation_1-auc:0.99762
[1743]	validation_0-auc:0.99902	validation_1-auc:0.99762
[1744]	validation_0-auc:0.99902

In [8]:
#阈值大概在0.2-0.4之间 本题对召回率较敏感，可适当降低一下阈值
thre = 0.25

#生成提交文件
sub = pd.DataFrame()
sub['hit_id'] = testpd['hit_id']
sub['flag_pred'] = y_pp_xgb
sub['event_id'] = testpd['event_id']
sub['flag_pred'] = sub['flag_pred'].apply(lambda x: 1 if x >= thre else 0)
sub.to_csv(os.path.join(pathf, "subsample.csv").format(sub['flag_pred'].mean()), index=False)
